### **IMPORTACIONES**

In [67]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from langchain_ollama import ChatOllama

### Variable de entorno

In [68]:
load_dotenv() 

api_key = os.getenv("GOOGLE_API_KEY")

### Subir PDF (PyPDFLoader)

In [69]:
def upload_pdf(file_path):
    try: 
        loader = PyPDFLoader(file_path)
        loader = loader.lazy_load()
        
        text = ""
        for page in loader:
            text += page.page_content + "\n"
        return text
    except Exception as e:
        print(f"Error al cargar el PDF: {e}")
        return []

### Separar contenido del PDF (CharacterTextSplitter)

In [70]:
def text_splitter(text):
    try:
        text_splitter = CharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=150,
            separator="\n",
        )
        texts = text_splitter.create_documents([text])
        return texts
    except Exception as e:
        print(f"Error al separar el texto: {e}")
        return []

### BD vectorial flexible (Chroma)

In [71]:
def get_vector_store(name_collection: str, embedding_model, persist_dir: str): 
    
    vector_store = Chroma(
        collection_name= name_collection,
        embedding_function=embedding_model,
        persist_directory=persist_dir     
    )    
    return vector_store

### Modelo original (nomic-embed-text)

In [72]:
embedding_original = OllamaEmbeddings(
    model = "nomic-embed-text"
)

### Vector Store del modelo original

In [73]:
vs_original = get_vector_store(
    name_collection="langchain",
    embedding_model=embedding_original, 
    persist_dir="./vectorstore"
)

### Búsqueda de contenido relevante según el input ingresado 

In [74]:
def retrieval(input_user: str, vector_store): 
    docs = vector_store.similarity_search(input_user)
    return docs

### Prompt (PromptTemplate)

In [75]:
prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre la energía solar y solo debes contestar si el contexto no está vacio.
    En caso de que no cuentes con la información solicitada responde "No cuento con información sobre eso".
    Utiliza siempre el contexto proporcionado para responder.
    contexto = {contexto}
    pregunta del usuario: {input_user}
""")

### Conexión con el modelo y respuesta final (ChatGoogleGenerativeAI)

In [76]:
def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.5-flash",
    temperature= 0.7
)

    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content

In [77]:
loader = upload_pdf("energia_solar.pdf")
texts = text_splitter(loader)

### Guardar el embedding original

In [78]:
vs_original.add_documents(texts)

['cb29182b-c96b-4ea1-b26b-83063c134301',
 'a9a1d54c-b339-48c9-9ae6-7c179f6e0adb',
 'a5741534-c6ab-4dc5-8f24-9b27cca08b7f']

---
## Punto 2: Experimento de Embeddings

In [79]:
embedding_nuevo = OllamaEmbeddings(
    model = "mxbai-embed-large"
)

### Vector Store del modelo nuevo

In [80]:
vs_nuevo = get_vector_store(
    name_collection="experimento_mxbai", 
    embedding_model=embedding_nuevo, 
    persist_dir="./vectorstore_2"
)

### Guardar el nuevo embedding

In [81]:
vs_nuevo.add_documents(texts)

['65a4b63d-f82d-4fb5-8276-b77688f68733',
 '269f0d59-c0b0-4b6a-b94f-b2e2fa648334',
 'bbb6343f-efae-44f5-89d2-c0296ad500d7']

### Comparación `nomic-embed-text` vs `mxbai-embed-large`

In [82]:
input_user = input("Human: ")
print(f"Pregunta: {input_user}")


# 1. Prueba con la BD ORIGINAL (nomic-embed-text)
print("\n--- 1. nomic-embed-text ---\n")
docs_original = retrieval(input_user=input_user, vector_store=vs_original)

print("\n[DOCS RECUPERADOS por 'nomic']:")
print([doc.page_content for doc in docs_original])

for chunk in response(input_user=input_user, contexto=docs_original):
    print(chunk, end=" ", flush=True)

# 2. Prueba con la BD NUEVA (mxbai-embed-large)
print("\n\n--- 2. mxbai-embed-large ---\n")
docs_nuevos = retrieval(input_user=input_user, vector_store=vs_nuevo)

print("\n[DOCS RECUPERADOS por 'mxbai']:")
print([doc.page_content for doc in docs_nuevos])

for chunk in response(input_user=input_user, contexto=docs_nuevos):
    print(chunk, end=" ", flush=True)

Pregunta: ¿Qué es la energía solar?

--- 1. nomic-embed-text ---


[DOCS RECUPERADOS por 'nomic']:
['La energía solar y su impacto ambiental \n \nLa energía solar es una fuente de energía renovable que se obtiene a partir de la \nradiación del sol. Esta forma de energía ha ganado popularidad en todo el mundo \ndebido a su potencial para reducir la dependencia de combustibles fósiles y \ndisminuir las emisiones de gases de efecto invernadero. \n \nLos sistemas de energía solar se dividen principalmente en dos categorías: \nenergía solar fotovoltaica y energía solar térmica. La energía fotovoltaica convierte \nla luz solar directamente en electricidad mediante el uso de células solares, \nmientras que la energía térmica utiliza colectores solares para calentar líquidos, \nque luego pueden ser usados para calefacción o generación de electricidad a \ntravés de turbinas de vapor. \n \nUno de los principales beneficios de la energía solar es su sostenibilidad. A \ndiferencia de los combustib

### Análisis de Resultados

 **Pregunta: "¿Qué es la energía solar?"**

 **`nomic-embed-text`:** Recuperó exitosamente el chunk con la definición.

 **`mxbai-embed-large`:** Falló, recuperando el chunk de la conclusión, que era irrelevante.

**Conclusión:** El modelo `nomic-embed-text` es claramente superior para este documento, demostrando una mejor capacidad de recuperación semántica tanto para la definición principal como para detalles específicos.

---
## Experimento de LLMs (Gemini vs. Llama 3)

### Conexión con Llama 3

In [ ]:
def response_llama(input_user: str, contexto: str):
    try:
        llm_local = ChatOllama(
            model="llama3", 
            temperature=0.7
            )
    except Exception as e:
        print(f"Error al conectar con Llama 3: {e}")
        yield from llm_local.stream(prompt.format(contexto=contexto, input_user=input_user)).content

### Comparación `Gemini` vs `Llama 3`

In [84]:
docs_nomic = docs_original # Los de 'nomic-embed-text'

print(f"Pregunta: {input_user}\n")
print(f"Contexto (Docs): {[doc.page_content for doc in docs_nomic]}\n")


# --- 1. Prueba con Google Gemini (API) ---
print("--- 1. Respuesta de Google Gemini (gemini-2.5-flash) ---")
for chunk in response(input_user=input_user, contexto=docs_nomic):
    print(chunk, end=" ", flush=True)


# --- 2. Prueba con Llama 3 (Local) ---
print("\n\n--- 2. Respuesta de Llama 3 (local) ---")
for chunk in response_llama(input_user=input_user, contexto=docs_nomic):
    print(chunk, end=" ", flush=True)

Pregunta: ¿Qué es la energía solar?

Contexto (Docs): ['La energía solar y su impacto ambiental \n \nLa energía solar es una fuente de energía renovable que se obtiene a partir de la \nradiación del sol. Esta forma de energía ha ganado popularidad en todo el mundo \ndebido a su potencial para reducir la dependencia de combustibles fósiles y \ndisminuir las emisiones de gases de efecto invernadero. \n \nLos sistemas de energía solar se dividen principalmente en dos categorías: \nenergía solar fotovoltaica y energía solar térmica. La energía fotovoltaica convierte \nla luz solar directamente en electricidad mediante el uso de células solares, \nmientras que la energía térmica utiliza colectores solares para calentar líquidos, \nque luego pueden ser usados para calefacción o generación de electricidad a \ntravés de turbinas de vapor. \n \nUno de los principales beneficios de la energía solar es su sostenibilidad. A \ndiferencia de los combustibles fósiles, la energía solar no produce emis

### Análisis de LLMs (Gemini vs. Llama 3)

Ambos modelos recibieron el mismo contexto de alta calidad (de `nomic-embed-text`) y el mismo prompt.

* **Gemini 2.5 Flash (API):** La respuesta fue muy rápida, concisa y **sintetizada**. Entendió la pregunta y extrajo solo la definición esencial.
* **Llama 3 (Local):** La respuesta fue tardada a pesar de ser local, fue más **extractiva**. En lugar de sintetizar, optó por copiar las primeras dos oraciones del contexto.

**Conclusión:** Para un chatbot que debe sonar natural, prefiero el comportamiento de **Gemini**, ya que sintetiza la información en lugar de simplemente copiarla. Sin embargo, **Llama 3** es una excelente alternativa gratuita y local que también encuentra la respuesta correcta.